In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [2]:
def logit(z): return 1./(1.+np.exp(-z))

In [3]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pystan'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return player1,player2,outcome
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [4]:
def prediction_accuracy(skill_data, samples, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(num_valid_game):
        i,j,result=valid_games[0][g],valid_games[1][g],valid_games[2][g]

        prediction = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()

        acc+= int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==0) )
    return acc/num_valid_game

In [5]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [6]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

In [7]:
scale=0.5
true_players_10 = generate_players(10,10)
train_games = generate_games(true_players_10, 1000, scale, style='pystan')

In [8]:
true_players_10

[8.79, 9.87, 4.87, 8.72, 8.76, 0.59, 3.41, 4.99, 2.6, 0.92]

In [9]:
skill_data = {
    'N': 10,
    'E': 1000,
    'scale': scale,
    'win':train_games[2],
    'PA': train_games[0],
    'PB': train_games[1]
}


Now, we can perform MCMC on the model, and extract the samples:

In [10]:
starting=time.time()
fit = sm.sampling(data=skill_data, iter=10000, chains=10)
print("spent:",round(time.time()-starting,3))

spent: 165.824


In [11]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [12]:
samples['skill'].mean(0)

array([ 2.66551628e+00,  4.10695920e+00, -5.54204655e-01,  3.15277326e+00,
        2.88488789e+00, -4.49688992e+00, -1.04155328e+00, -3.10944401e-03,
       -2.00200745e+00, -4.73098490e+00])

In [13]:
prediction_accuracy(skill_data, samples, true_players_10, num_valid_game=1000, scale=scale)

0.83

In [15]:

scale=0.5
num_of_players=10
result={}
for num_of_games in [10,50,100,500,700,1000,3000,5000]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,10)
        train_games = generate_games(true_players, num_of_games, scale, style='pystan')
        print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=num_of_players)
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy(skill_data, samples, true_players, num_valid_game=10000, scale=scale)
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result)

starting num_of_games = 10
true skills: [5.59, 4.14, 5.8, 5.38, 3.4, 10.0, 4.15, 1.9, 9.94, 1.53]
spent: 8.439
predicted skills: [ 1.73996712  0.53845243  0.78672853 -1.62510683 -2.31110893  1.62384057
 -0.72709902  1.77120029  0.00262405 -1.82774235]
acc: 0.5687 

true skills: [9.52, 0.2, 6.92, 7.73, 3.04, 3.33, 4.73, 5.66, 6.48, 5.25]
spent: 7.683
predicted skills: [ 3.32273508 -2.96897479  0.69435515 -0.01054685  1.87654417 -1.17689575
  0.14246377 -0.00375831  0.92194919 -2.84192807]
acc: 0.648 

true skills: [5.07, 9.64, 0.79, 7.0, 6.92, 8.75, 7.43, 1.43, 2.0, 4.6]
spent: 8.273
predicted skills: [ 0.00454012  2.1421007   0.0093289  -2.6035516  -0.62142934  2.00901789
  2.62851695 -1.55628329 -2.04544386  0.0146242 ]
acc: 0.6401 

true skills: [5.02, 9.49, 9.19, 1.8, 8.93, 6.46, 1.46, 5.31, 5.61, 6.46]
spent: 8.602
predicted skills: [-1.14338181e+00 -2.42304417e-03  2.72224162e+00 -1.45510457e+00
  1.91450141e+00  5.62988856e-01  9.72175442e-01  6.16764075e-01
 -2.73592063e+00 -1.4

acc: 0.7801 

true skills: [1.58, 9.6, 1.73, 5.92, 6.3, 0.46, 0.09, 7.61, 5.88, 6.53]
spent: 97.487
predicted skills: [-3.41589733  4.5079393  -2.91263313  0.89359308  1.45042443 -3.52101578
 -4.00176519  2.88292313  1.90548137  2.07193751]
acc: 0.808 

true skills: [8.19, 7.09, 0.97, 0.69, 8.59, 6.62, 7.6, 9.31, 6.17, 4.1]
spent: 98.366
predicted skills: [ 2.05376522  1.1802357  -5.45315316 -4.64771268  3.58601559  0.6830962
  1.46269307  3.37383444 -0.2048254  -2.02744285]
acc: 0.7798 

true skills: [1.41, 5.66, 6.19, 4.11, 6.7, 5.35, 8.79, 3.64, 7.58, 1.12]
spent: 93.627
predicted skills: [-4.60132333  0.41822118  1.30981666 -1.61014296  2.23981769  0.43104377
  4.26367822 -1.21219568  2.51975524 -3.84466656]
acc: 0.7681 

true skills: [7.74, 8.38, 2.98, 4.48, 9.88, 9.44, 5.79, 3.88, 4.63, 2.66]
spent: 117.706
predicted skills: [ 1.54173235  2.80562076 -2.3471605  -1.08842623  3.2140141   3.46966994
 -0.87280217 -2.20102314 -1.13312047 -3.23172925]
acc: 0.7867 

true skills: [6.53, 

spent: 627.363
predicted skills: [ 0.80577944 -3.67489653 -1.54432387  0.82298701  2.28142766  0.7434816
  4.9717826  -3.40285287  1.87329202 -2.85049114]
acc: 0.7892 

true skills: [4.2, 5.79, 3.55, 1.32, 0.77, 8.94, 4.48, 3.17, 5.18, 7.16]
spent: 667.951
predicted skills: [-0.08726111  1.35243165 -0.65370572 -3.67894081 -3.73662724  4.80019398
  0.17233933 -1.53898692  0.68781383  2.54056222]
acc: 0.7766 

true skills: [7.26, 5.23, 9.03, 0.76, 2.12, 2.66, 4.59, 4.63, 4.35, 8.25]
spent: 627.635
predicted skills: [ 2.31760273  0.28535276  4.0413585  -3.67135773 -2.83164829 -1.89504136
 -0.67227058 -0.42473289 -0.7614827   3.46186067]
acc: 0.7856 

true skills: [7.25, 9.99, 6.74, 9.01, 6.92, 3.41, 4.63, 6.96, 4.7, 1.39]
spent: 643.374
predicted skills: [ 0.92790129  3.6188931   0.76276753  2.95741993  0.94459167 -2.4853569
 -1.66381354  0.84220162 -1.45148651 -4.38458378]
acc: 0.7672 

Complete 3000 | Spent: -7204.92
starting num_of_games = 5000
true skills: [8.63, 3.93, 6.65, 5.99, 8.2

In [16]:
print(result)

{10: [0.5687, 0.648, 0.6401, 0.5831, 0.6393, 0.5806, 0.5446, 0.6709, 0.6507, 0.6748], 50: [0.7424, 0.7751, 0.7941, 0.7598, 0.8221, 0.7576, 0.6775, 0.7856, 0.7673, 0.7223], 100: [0.7981, 0.7779, 0.7916, 0.7767, 0.8005, 0.7784, 0.7433, 0.8229, 0.6958, 0.8054], 500: [0.8349, 0.8185, 0.7801, 0.808, 0.7798, 0.7681, 0.7867, 0.8254, 0.7608, 0.8039], 700: [0.7561, 0.82, 0.8038, 0.8209, 0.7846, 0.7745, 0.7113, 0.7389, 0.7993, 0.8121], 1000: [0.8018, 0.8118, 0.7797, 0.8009, 0.8021, 0.8123, 0.7984, 0.8289, 0.7967, 0.7907], 3000: [0.8325, 0.7493, 0.764, 0.8042, 0.8141, 0.816, 0.7892, 0.7766, 0.7856, 0.7672], 5000: [0.8056, 0.755, 0.8069, 0.7964, 0.7964, 0.7395, 0.8112, 0.78, 0.6649, 0.7627]}


In [17]:
def avg(x):
    return sum(x)/len(x)
for i in result:
    print(i,avg(result[i]))

10 0.62008
50 0.76038
100 0.77906
500 0.79662
700 0.7821499999999999
1000 0.8023299999999999
3000 0.7898699999999999
5000 0.77186


In [ ]:

scale=0.5
num_of_players=50
result={}
for num_of_games in [10,50,100,500,700,1000,3000,5000]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,10)
        train_games = generate_games(true_players, num_of_games, scale, style='pystan')
#         print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=num_of_players)
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
#         print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy(skill_data, samples, true_players, num_valid_game=10000, scale=scale)
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result)

starting num_of_games = 10
spent: 72.264
acc: 0.5381 

spent: 79.36
acc: 0.5324 

spent: 80.057
acc: 0.5616 

spent: 125.539
acc: 0.5644 

spent: 121.672
acc: 0.5488 

spent: 128.133
acc: 0.5416 

spent: 113.862
acc: 0.5283 

spent: 132.592
acc: 0.5462 

spent: 134.456
acc: 0.5375 

spent: 134.175
acc: 0.5558 

Complete 10 | Spent: 3163.67
starting num_of_games = 50
spent: 222.258
acc: 0.6468 

spent: 187.871
acc: 0.6576 

spent: 195.202
acc: 0.6379 



In [ ]:
print(result)

In [ ]:
def avg(x):
    return sum(x)/len(x)
for i in result:
    print(i,avg(result[i]))